In [1]:
import time
import flappy_bird_gym
import gymnasium
env = flappy_bird_gym.make("FlappyBird-rgb-v0")

C:\Users\sultan.yarylgasimov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, Resizing
from keras_cv.layers import Grayscale

def build_model(obs, actions):
    model = Sequential()
    model.add(Reshape(obs, input_shape=(1, obs[0], obs[1], obs[2])))
    model.add(Grayscale(output_channels=1))
    model.add(Resizing(80, 80, interpolation="bilinear", crop_to_aspect_ratio=False))
    model.add(Conv2D(32, (8, 8), activation='relu', strides=4))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (4, 4), activation='relu', strides=2))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='ReLU'))
    model.add(Dense(actions, activation='linear'))
    model.summary()
    return model

In [36]:
obs = env.observation_space.shape
actions = env.action_space.n

In [37]:
env.observation_space.shape

(288, 512, 3)

In [38]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [39]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=0.1, value_min=.0001, value_test=.0, nb_steps=1000000)
    memory = SequentialMemory(limit=100000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                enable_dueling_network=True, dueling_type='avg',
                nb_actions=actions, nb_steps_warmup=500)
    return dqn

In [40]:
model = build_model(obs, actions)
dqn = build_agent(model, actions)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 288, 512, 3)       0         
                                                                 
 grayscale_4 (Grayscale)     (None, 288, 512, 1)       0         
                                                                 
 resizing_4 (Resizing)       (None, 80, 80, 1)         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 19, 19, 32)        2080      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 9, 9, 32)         0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 3, 3, 64)          32832     
                                                      

In [41]:
import warnings
warnings.simplefilter("ignore")

In [42]:
from tensorflow.keras.optimizers.legacy import Adam
#Training the Neural Network
dqn.compile(Adam(learning_rate=0.0001))

In [43]:
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=1)

Training for 1000000 steps ...
Interval 1 (0 steps performed)
  584/10000 [>.............................] - ETA: 26:13 - reward: -10.2740done, took 98.743 seconds


In [44]:
#Saving weights of Neural Network
dqn.save_weights("weights/flappy_bird_solution_rgb.h5", overwrite=True)

In [ ]:
dqn.load_weights("weights/flappy_bird_solution_rgb.h5")

In [ ]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
